In [ ]:
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from sneks_ai import SnakeGame, Snake, Apple  # use the classes from sneks-ai.py

In [ ]:
game = SnakeGame(
    canvas_size=20, 
    nApples=2, 
    nSnakes =2
    )
snakeIDs = [snake for snake in game.snakes]
plt.figure(figsize=(10,10))
plt.imshow(game.get_state(snakeID=snakeIDs[0]))
plt.xticks([])
plt.yticks([])
plt.show()

# Outside of the game you request actions of each snake, and then update the game with the action_vector, where the action vector length is the number of snakes

In [ ]:
# Outside of the game you request actions of each snake, and then update the game with the action_vector, where the action vector length is the number of snakes
# E.G.
numSnakes = 3
game = SnakeGame(nSnakes = numSnakes)
while not game.done:
    # initialize the action vector
    action_vector = [None]*numSnakes
    for snake in game.snakes:
        if game.snakes[snake].alive:
            action_vector[snake] = game.snakes[snake].take_action(game.get_state(snakeID=snake), game.choices)
    # update the game
    game.updateGame(action_vector)